In [1]:
import DirectDmTargets as dddm

calculating effective efficiency for Xe detector done
calculating effective efficiency for Ar detector done
calculating effective efficiency for Ge detector done


In [13]:
import corner

In [14]:
import matplotlib.pyplot as plt

In [15]:
import numpy as np

In [16]:
import os

In [17]:
def load_chain(item='latest'):
    base = 'results/'
    save = 'test'
    files = os.listdir(base)
    if item is 'latest':
        item = max([int(f.split(save)[-1]) for f in files])
    result = {}
    load_dir = base + save + str(item) + '/'
    if not os.path.exists(load_dir):
        raise FileNotFoundError(f"Cannot find {load_dir} specified by arg: "
                                f"{item}")
    print("loading", load_dir)

    keys = ['config', 'full_chain', 'flat_chain']

    for key in keys:
        result[key] = np.load(load_dir + key + '.npy', allow_pickle=True)
    print(f"done loading\naccess result with:\n{keys}")
    return result

In [18]:
results = {}
for i in range(200):
    try:
        results[i] = load_chain(i)
    except FileNotFoundError:
        print(f"No result for {i}")
    print()

No result for 0

No result for 1

loading results/test2/
done loading
access result with:
['config', 'full_chain', 'flat_chain']

No result for 3

No result for 4

loading results/test5/
done loading
access result with:
['config', 'full_chain', 'flat_chain']

loading results/test6/
done loading
access result with:
['config', 'full_chain', 'flat_chain']

loading results/test7/
done loading
access result with:
['config', 'full_chain', 'flat_chain']

No result for 8

No result for 9

No result for 10

No result for 11

No result for 12

No result for 13

No result for 14

No result for 15

No result for 16

No result for 17

No result for 18

No result for 19

loading results/test20/
done loading
access result with:
['config', 'full_chain', 'flat_chain']

loading results/test21/
done loading
access result with:
['config', 'full_chain', 'flat_chain']

No result for 22

No result for 23

No result for 24

No result for 25

No result for 26

No result for 27

No result for 28

loading result

In [19]:
results.keys()

dict_keys([2, 5, 6, 7, 20, 21, 29, 33, 35, 37, 66, 71, 72, 78, 79, 87, 90, 91, 94, 95, 97, 99, 120, 121, 123, 129, 130, 131, 132, 134, 135, 136, 146, 153, 158, 162])

In [ ]:
for key in results.keys():
    print(f"---{key}---")
    result = results[key]
    try:       
        info = "$M_\chi}$=" + f"{result['config'].item()['mw']}"
        for prior_key in result['config'].item()['prior'].keys():
            try:
                mean = result['config'].item()['prior'][prior_key]['mean']
                info += f"\n{prior_key} = {mean}"
            except KeyError:
                pass
         
        nwalkers, nsteps, ndim = np.shape(result['full_chain'])
        
        info += "\npoisson = %s"%result['config'].item()['poisson']   
        info += "\nnwalkers = %s"%nwalkers
        info += "\nnsteps = %s"%nsteps
        for str_inf in ['notes', 'start']:
            try: 
                info += f"\n{str_inf} = %s"%result['config'].item()[str_inf]   
            except KeyError:
                pass
#         labels = ['log_mass',
#                         'log_cross_section',
#                         'v_0',
#                         'v_esc',
#                         'density'][:ndim]
        labels = ['mass',
                        'cross_section',
                        'v_0',
                        'v_esc',
                        'density'][:ndim]
        truths = [result['config'].item()['mw'],
                  result['config'].item()['sigma'],
                  result['config'].item()['v_0'],
                  result['config'].item()['v_esc'],
                  result['config'].item()['rho_0']
                  ][:ndim]
        
        fig, axes = plt.subplots(len(labels), figsize=(10, 5), sharex=True)
        samples = result['full_chain']
        for i in range(len(labels)):
            ax = axes[i]
            ax.plot(samples[:, :, i], "k", alpha=0.3)
            ax.axhline(truths[i])
            ax.set_xlim(0, len(samples))
            ax.set_ylabel(labels[i])
            ax.yaxis.set_label_coords(-0.1, 0.5)
            
        axes[-1].set_xlabel("step number")
        plt.savefig(f"results/test{key}/walkers{key}.png", dpi = 200)
        plt.show()
        
        fig= corner.corner(result['flat_chain'],
                            labels = labels,
                            truths = truths,
#                             quantiles=[0.05, 0.5, 0.95],
                           show_titles=True
                           );
#         fig.suptitle(f"Fit {key}\n\n\n")
        fig.axes[1].set_title(f"Fit {key}", loc = 'left')
        fig.axes[1].text(0,1,info, verticalalignment = 'top')
        plt.savefig(f"results/test{key}/corner{key}.png", dpi = 200)
        plt.show()
        
    except AssertionError:
        pass
    except ValueError:
        print("In one of the dimentions the walkers did not walk resulting in an empty range. This is not good, delete threse results.")